In [ ]:
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 60.5 MB/s eta 0:00:00


In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2023-10-16 12:24:54--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M   349KB/s    in 2m 2s   

2023-10-16 12:26:57 (362 KB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [ ]:
!gunzip reviews_Cell_Phones_and_Accessories_5.json.gz

In [ ]:
import gensim
import pandas as pd

In [ ]:
# Download the dataset from https://www.kaggle.com/datasets/zainabhaddad/reviews-cell-phones-and-accessories-5/
df = pd.read_json("/content/reviews_Cell_Phones_and_Accessories_5.json",lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [ ]:
df.shape

(194439, 9)

# Preprocessing & Tokenization

In [ ]:
df['reviewText'][0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [ ]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [ ]:
review_text = df['reviewText'].apply(gensim.utils.simple_preprocess)
review_text.head()

0    [they, look, good, and, stick, good, just, don...
1    [these, stickers, work, like, the, review, say...
2    [these, are, awesome, and, make, my, phone, lo...
3    [item, arrived, in, great, time, and, was, in,...
4    [awesome, stays, on, and, looks, great, can, b...
Name: reviewText, dtype: object

# Train the Word2Vec Model

In [ ]:
# for preparing the data we have to specify the window size
model = gensim.models.Word2Vec(window=10,
                               vector_size=100, # what should be the size of my feature vector
                               min_count=2, # we will consider any word only if it used atleast twice
                               sg=1) #appplying the skipgram model on the given data

# Build the Vocabulory

In [ ]:
model.build_vocab(review_text)

In [ ]:
model.epochs #default value

5

In [ ]:
model.window # length of my sliding window

10

In [ ]:
model.vector_size

100

# Train Word2Vec Model

In [ ]:
model.train(review_text, total_examples=model.corpus_count, epochs = model.epochs)

(61509215, 83868975)

In [ ]:
model.corpus_count #total corpus count we have

194439

# Save the Model

In [ ]:
model.save("./DT_word2vec.model")

# Find the Similar Words and Similarity between the words

In [ ]:
# Word Embeddings
model.wv.most_similar("bad")

[('terrible', 0.7252035737037659),
 ('horrible', 0.6947677731513977),
 ('ok', 0.671481192111969),
 ('okay', 0.6648902893066406),
 ('bellow', 0.6609990000724792),
 ('good', 0.660744309425354),
 ('execpt', 0.6577576994895935),
 ('coulda', 0.6543023586273193),
 ('miffed', 0.6541720032691956),
 ('goodthe', 0.6452839374542236)]

In [ ]:
#computing the similarity
model.wv.similarity(w1='cheap',w2 = 'inexpensive')

0.63017863

In [ ]:
matrix = model.wv.vectors # extract the word embedding matrix

print(f"shape of the word embedding matrix is {matrix.shape}")

shape of the word embedding matrix is (35561, 100)


In [ ]:
W_out = model.syn1neg
W_out.shape # Output weights

(35561, 100)

In [ ]:
#https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4
import random
random_word = random.choice(model.wv.index_to_key)
random_word

'twisted'

In [ ]:
vocab_list = list(model.wv.index_to_key)

In [ ]:
len(vocab_list)

35561

In [ ]:
# vocab_list

In [ ]:
model.wv

In [ ]:
vocab_len = len(vocab_list)

In [ ]:
# model.wv.index_to_key

In [ ]:
# model.wv.key_to_index

In [ ]:
vocab_int2word = {model.wv.key_to_index[w]:model.wv.get_vector(w, norm=True) for w in vocab_list}

In [ ]:
vocab_int2word

{0: array([ 0.11262804,  0.28983334,  0.12957579, -0.03236028, -0.02756411,
        -0.17741868, -0.05550884,  0.14925225, -0.13983771, -0.10798392,
         0.00910565, -0.03408887, -0.10036407,  0.03728202, -0.0817401 ,
        -0.10374094, -0.02099307,  0.21243782, -0.04219551, -0.09129383,
         0.08363278,  0.00140036,  0.08739972,  0.0442976 ,  0.02879091,
         0.02933109, -0.09427904,  0.11767397, -0.1826429 ,  0.04103651,
         0.01082103, -0.11554097, -0.087706  , -0.2744233 ,  0.0015518 ,
         0.17445649,  0.01808567,  0.05599356,  0.01914574, -0.16454092,
        -0.00040177, -0.01140637, -0.10716001,  0.0085213 ,  0.00827788,
        -0.10765335, -0.15014645, -0.02734755, -0.12854117,  0.20146775,
         0.11075696, -0.06037388,  0.03718673,  0.01342222, -0.10875163,
         0.07930153,  0.01143924,  0.07310493, -0.05495999,  0.13114294,
        -0.01178083, -0.01410668,  0.06141902,  0.06508593, -0.00919682,
         0.18980834,  0.07985537,  0.02040885, -

In [ ]:
vocab_int2word[4697]

array([ 0.14229509,  0.1934074 ,  0.0671732 , -0.04045045,  0.13793278,
       -0.01801313,  0.15018493,  0.15249132,  0.00894223,  0.09886851,
       -0.19554563, -0.22337896,  0.03962352, -0.02681297,  0.01546367,
        0.1950367 ,  0.17875375, -0.06580887, -0.01699857, -0.02484909,
        0.02784138, -0.04373717,  0.105397  , -0.01655175,  0.05053085,
        0.0205068 , -0.05646446, -0.15955853, -0.14408898,  0.11880438,
        0.10734172,  0.09179767,  0.20084853, -0.18946634, -0.0515818 ,
        0.08752993,  0.13045189,  0.00269543,  0.07452065, -0.13925257,
        0.03848657, -0.11540106,  0.03533496,  0.13537991, -0.01903331,
        0.04878248, -0.01455881, -0.0422862 , -0.03344366,  0.03316685,
        0.01383037, -0.17996827,  0.05859092,  0.1028441 , -0.1736695 ,
       -0.08021048,  0.05369551, -0.15524566, -0.14293036,  0.05781311,
       -0.08100726,  0.02518144, -0.06358559,  0.05377617, -0.08718412,
        0.06886491, -0.01514733, -0.002322  , -0.10941087, -0.00

# Forward Pass

In [ ]:
import numpy as np
one_hot = np.zeros(shape=(1,vocab_len))

one_hot[0,model.wv.key_to_index['confidence']] = 1

In [ ]:
# performing the dot product
z_proj = np.dot(one_hot,matrix)
z_proj.shape

(1, 100)

In [ ]:
# Multiplying word embedding representation and W_out
a_proj = z_proj #taking the transpose of W_ouy

z_out = np.dot(a_proj,W_out.T)
z_out.shape

(1, 35561)

In [ ]:
#Softmax activation function
a_out = np.exp(z_out)/np.sum(np.exp(z_out))

In [ ]:
a_out.shape

(1, 35561)

In [ ]:
a_out = a_out.flatten() # performing flattening operation

indices = np.argsort(a_out) #sort in ascending order
indices = indices[::-1] # sort in descending order

for i in range(10):
  word_index = indices[i]
  word = model.wv.index_to_key[word_index]
  a_out_i = a_out[word_index]

  print(f"{word} with score {a_out_i}")

confidence with score 0.004525855668288994
confident with score 0.0017426705430831537
provide with score 0.0009793082343290752
survive with score 0.0009054389128522058
solid with score 0.000761887986227899
trust with score 0.000721625623535075
absorb with score 0.0006799760896676241
gives with score 0.0006659830255040383
peace with score 0.0006505746366579717
provides with score 0.0006029995606287925
